# Lambda notebook demo v. 1.0.2

## Author: Kyle Rawlins

This notebook provides a demo of the core capabilities of the lambda notebook, aimed at linguists who already have training in semantics (but not necessarily implemented semantics).

Last updated Aug 2023. Version history:

 * 0.5: first version
 * 0.6: updated to work with refactored class hierarchy (Apr 2013)
 * 0.6.1: small fixes to adapt to changes in various places (Sep 2013)
 * 0.7: various fixes to work with alpha release (Jan 2014)
 * 0.9: substantial updates, merge content from LSA poster (Apr 2014)
 * 0.95: substantial updates for a series of demos in Apr-May 2014
 * 1.0: various changes / fixes, more stand-alone text (2017)
 * 1.0.1: small tweaks (Nov/Dec 2018)
 * 1.0.2: bugfixes
 
To run through this demo incrementally, use shift-enter (runs and moves to next cell).  If you run things out of order, you may encounter problems (missing variables etc.)

In [ ]:
from lamb.types import TypeMismatch, type_e, type_t, type_property
from lamb.meta import TypedTerm, TypedExpr, LFun, CustomTerm

In [ ]:
# There are several ways of display outputs, this one is the default:
lamb.display.default(style=lamb.display.DerivStyle.BOXES)
# you can also try:
# lamb.display.default(style=lamb.display.DerivStyle.PROOF)

# First pitch

Have you ever wanted to type something like this in, and have it actually do something?

In [ ]:
%%lamb
||every|| = λ f_<e,t> : λ g_<e,t> : Forall x_e : f(x) >> g(x)
||student|| = L x_e : Student(x)
||danced|| = L x_e : Danced(x)

In [ ]:
r = ((every * student) * danced)
r

In [ ]:
r.tree()

# Two problems in formal semantics #

1. Type-driven computation could be a lot easier to visualize and check.  (Q: could it be made too easy?)

2. Grammar fragments as in Montague Grammar: good idea in principle, hard to use in practice.

  * A **fragment** is a *complete* formalization of *sublanguage* consisting of the *key relevant phenomena* for the problem at hand.  (Potential problem-points italicized.)

Solution: a system for developing interactive fragments: "*IPython Lambda Notebook*"

* Creator can work interactively with analysis -- accelerate development, limit time spent on tedious details.
* Reader can explore derivations in ways that are not typically possible in typical paper format.
* Creator and reader can be certain that derivations work, verified by the system.
* Bring closer together formal semantics and computational modeling.

Inspired by:

 * Von Eijck and Unger (2013): implementation of compositional semantics in Haskell.  No interface (beyond standard Haskell terminal); great if you like Haskell.  Introduced the idea of a fragment in digital form.
 * UPenn Lambda calculator (Champollion, Tauberer, and Romero 2007): teaching oriented.  (Now under development again.)
 * `nltk.sem`: implementation of the lambda calculus with a typed metalanguage, interface with theorem provers.  No interactive interface.
 * Jealousy of R studio, Matlab, Mathematica, etc.

### The role of formalism & fragments ###

What does *formal* mean in semantics?  What properties should a theory have?

 1. Mathematically precise (lambda calculus, type theory, logic, model theory(?), ...)
 2. Complete (covers "all" the relevant data).
 3. Predictive (like any scientific theory).
 4. Consistent, or at least compatible (with itself, analyses of other phenomena, some unifying conception of the grammar).
 
The *method of fragments* (Partee 1979, Partee and Hendriks 1997) provides a structure for meeting these criteria.

 * Paper with a fragment provides a working system.  (Probably.)
 * Explicit outer bound for empirical coverage.
 * Integration with a particular theory of grammar.  (To some extent.)
 * Explicit answer to many detailed questions not necessarily dealt with in the text.
 
**Claim**: fragments are a method of replicability, similar to a computational modeller providing their model.

 * To be clear, a fragment is neither necessary nor sufficient for having a good theory / analysis / paper...

Additional benefit: useful internal check for researcher.

>"...But I feel strongly that it is important to try to [work with fully explicit fragments] periodically, because otherwise it is extremely easy to think that you have a solution to a problem when in fact you don't." (Partee 1979, p. 41)

### The challenges of fragments

Part 1 of the above quote:

>"It can be very frustrating to try to specify frameworks and fragments explicitly; this project has not been entirely rewarding.  I would not recommend that one always work with the constraint of full explicitness." (Ibid.)

 * Fragments can be tedious and time-consuming to write (not to mention hard).
 * Fragments as traditionally written are in practice not easy for a reader to use.
 
   - Dense/unapproachable.  With exactness can come a huge chunk of hard-to-digest formalism.  E.g. Partee (1979), about 10% of the paper.
   - Monolithic/non-modular.  For the specified sublanguage, everything specified.  Outside the bounds of the sublanguage, nothing specified.  How does the theory fit in with others?
   - Exact opposite of the modern method -- researchers typically hold most aspects of the grammar constant (implicitly) while changing a few key points.  (*Portner and Partee intro*)

**Summary:** In practice, the typical payoff for neither the reader nor the writer of a fragment exceeded the effort.


# A solution: digital fragments

Von Eijck and Unger 2010: specify a fragment in digital form.

* They use Haskell.  Type system of Haskell extremely well-suited to natural language semantics.
* (Provocative statement) Interface, learning curve of Haskell not well suited to semanticists (or most people)?

### Benefits of digital fragments (in principle)

* Interactive.
* Easy to distribute, adapt, modify.
* Possibility of modularity.  (E.g. abstract a 'library' for compositional systems away from the analysis of a particular phenomenon.)
* Bring closer together the CogSci idea of a 'computational model' to the project of natural language semantics.
* Connections to computational semantics. (weak..)

### What sorts of things might we want in a fragment / system for fragments?

* Typed lambda calculus.
* Logic / logical metalanguage.
* Framework for semantic composition.  (Broad...)
* Model theory? (x)
* Interface with theorem provers? (x)

IPython Lambda Notebook aims to provide these tools in a usable, interactive, format.

* Choose Python, rather than Haskell/Java.  Easy learning curve, rapid prototyping, existence of IPython.

**Layer 1**: interface using IPython Notebook.

**Layer 2**: flexible typed metalanguage.

**Layer 3**: composition system for object language, building on layer 2.

## Layer 1: an interface using IPython/Jupyter Notebook (Perez and Granger 2007) ##

 * Client-server system where a specialized IPython "kernel" is running in the background. This kernel implements various tools for formal semantics (see parts 2-3).
 * Page broken down into cells in which can be entered python code, markdown code, raw text, other formats.
 * Jupyter: supports display of graphical representations of python objects.
 * Notebook format uses the "MathJax" framework to enable it to render most math-mode latex.  Can have python objects automatically generate decent-looking formulas.  Can use latex math mode in documentation as well (e.g. $\lambda x \in D_e : \mathit{CAT}(x)$)

This all basically worked off-the-shelf.

* Bulk of interface work so far: rendering code for logical and compositional representations.
* Future: interactive widgets, etc.

In [ ]:
meta.pmw_test1

In [ ]:
meta.pmw_test1._repr_latex_()

&nbsp;

## Part 2: a typed metalanguage ##

The **metalanguage** infrastructure is a set of classes that implement the building blocks of logical expressions, lambda terms, and various combinations combinations. This rests on an implementation of a **type system** that matches what semanticists tend to assume.

Starting point (2012): a few implementations of things like predicate logic do exist, this is an intro AI exercise sometimes.  I started with the [AIMA python](http://code.google.com/p/aima-python/) _Expr_ class, based on the standard Russell and Norvig AI text.  But, had to scrap most of it.  Another starting point would have been `nltk.sem` (I was unaware of its existence at the time.)

Preface cell with `%%lamb` to enter metalanguage formulas directly.  The following cell defines a variable `x` that has type e, and exports it to the notebook's environment.

In [ ]:
%%lamb reset
x = x_e # define x to have this type

In [ ]:
x.type

This next cell defines some variables whose values are more complex object -- in fact, functions in the typed lambda calculus.

In [ ]:
%%lamb
test1 = L p_t : L x_e : P(x) & p # based on a Partee et al example
test1b = L x_e : P(x) & Q(x)
t2 = Q(x_e)

These are now registered as variables in the python namespace and can be manipulated directly.  A typed lambda calculus is fully implemented with all that that entails -- e.g. the value of `test1` includes the whole syntactic structure of the formula, its type, etc. and can be used in constructing new formulas.  The following cells build a complex function-argument formula, and following that, does the reduction.

(Notice that beta reduction works properly, i.e. bound $x$ in the function is renamed in order to avoid collision with the free `x` in the argument.)

In [ ]:
test1(t2)

In [ ]:
test1(t2).reduce()

In [ ]:
%%lamb
catf = L x_e: Cat(x)
dogf = λx: Dog(x_e)

In [ ]:
(catf(x)).type

In [ ]:
catf.type

Type checking of course is a part of all this.  If the types don't match, the computation will throw a `TypeMismatch` exception.  The following cell uses python syntax to catch and print such errors.

In [ ]:
with lamb.errors():
    display(test1(x)) # function is type <t,<e,t>> so will trigger a type mismatch.

A more complex expression:

In [ ]:
%%lamb
p2 = (Cat_<e,t>(x_e) & p_t) >> (Exists y: Dog_<e,t>(y_e))

What is going on behind the scenes?  The objects manipulated are recursively structured python objects of class TypedExpr.

Class _TypedExpr_: parent class for typed expressions.  Key subclasses:

* BinaryOpExpr: parent class for things like conjunction.
* TypedTerm: variables, constants of arbitrary type
* BindingOp: operators that bind a single variable
  * LFun: lambda expression

Many straightforward expressions can be parsed.  Most expressions are created using a call to TypedExpr.factory, which is abbreviated as "te" in the following examples.  The `%%lamb` magic is calling this behind the scenes.

Three ways of instantiating a variable `x` of type `e`:

In [ ]:
%%lamb 
x = x_e # use cell magic

In [ ]:
x = te("x_e") # use factory function to parse string
x

In [ ]:
x = meta.TypedTerm("x", types.type_e) # use object constructer
x

Various convenience python operators are overloaded, including functional calls.  Here is an example repeated from earlier in two forms:

In [ ]:
%%lamb
p2 = (Cat_<e,t>(x_e) & p_t) >> (Exists y: Dog_<e,t>(y_e))

In [ ]:
p2 = (te("Cat_<e,t>(x)") & te("p_t")) >> te("(Exists y: Dog_<e,t>(y_e))")
p2

Let's examine in detail what happens when a function and argument combine.

In [ ]:
catf = meta.LFun(types.type_e, te("Cat(x_e)"), "x")
catf

In [ ]:
catf(te("y_e"))

Building a function-argument expression builds a complex, unreduced expression.  This can be explicitly reduced (note that the `reduce_all()` function would be used to apply reduction  recursively):

In [ ]:
catf(te("y_e")).reduce()

In [ ]:
(catf(te("y_e")).reduce()).derivation

The metalanguage supports some basic type inference.  Type inference happens already on combination of a function and argument into an unreduced expression, not on beta-reduction.

In [ ]:
%lamb ttest = L x_X : P_<?,t>(x) # type <?,t>
%lamb tvar = y_t
ttest(tvar)

&nbsp;

## Part 3: composition systems for an object language ##

On top of the metalanguage are '**composition systems**' for modeling (step-by-step) semantic composition in an object language such as English.  This is the part of the lambda notebook that tracks and manipulates mappings between object language elements (words, trees, etc) and denotations in the metalanguage.  

A composition at its core consists of a set of composition rules; the following cell defines a simple composition system that will be familiar to anyone who has taken a basic course in compositional semantics.  (This example is just a redefinition of the default composition system.)

In [ ]:
# none of this is strictly necessary, the built-in library already provides effectively this system.
fa = lang.BinaryCompositionOp("FA", lang.fa_fun, reduce=True)
pm = lang.BinaryCompositionOp("PM", lang.pm_fun, commutative=False, reduce=True)
pa = lang.BinaryCompositionOp("PA", lang.pa_fun, allow_none=True)
demo_hk_system = lang.CompositionSystem(name="demo system", rules=[fa, pm, pa])
lang.set_system(demo_hk_system)
demo_hk_system

Expressing denotations is done in a `%%lamb` cell, and almost always begins with lexical items.  The following cell defines several lexical items that will be familiar from introductory exercises in the Heim & Kratzer 1998 textbook "Semantics in Generative Grammar".  These definitions produce items that are subclasses of the class `Composable`.

In [ ]:
%%lamb
||cat|| = L x_e: Cat(x)
||gray|| = L x_e: Gray(x)
||john|| = John_e
||julius|| = Julius_e
||inP|| = L x_e : L y_e : In(y, x) # `in` is a reserved word in python
||texas|| = Texas_e
||isV|| = L p_<e,t> : p # `is` is a reserved word in python

In the purely type-driven mode, composition is triggered by using the '`*`' operator on a `Composable`.  This searches over the available composition operations in the system to see if any results can be had.  `inP` and `texas` above should be able to compose using the FA rule:

In [ ]:
inP * texas

On the other hand `isV` is looking for a property, so we shouldn't expect succesful composition.  Below this I have given a complete sentence and shown some introspection on that composition result.

In [ ]:
julius * isV # will fail due to type mismatches

In [ ]:
sentence1 = julius * (isV * (inP * texas))
sentence1

In [ ]:
sentence1.trace()

Composition will find all possible paths (beware of combinatorial explosion).  I have temporarily disabled the fact that standard PM is symmetric/commutative (because of conjunction), to illustrate a case with multiple composition paths:

In [ ]:
gray * cat

In [ ]:
gray * (cat * (inP * texas))

In [ ]:
a = lang.Item("a", isV.content) # identity function for copula as well
isV * (a * (gray * cat * (inP * texas)))

In [ ]:
np = ((gray * cat) * (inP * texas))
vp = (isV * (a * np))
sentence2 = julius * vp
sentence2

In [ ]:
sentence1.results[0]

In [ ]:
sentence1.results[0].tree()

In [ ]:
sentence2.results[0].tree()

One of the infamous exercise examples from Heim and Kratzer (names different):

    (1) Julius is a gray cat in Texas fond of John.
 
First let's get rid of all the extra readings, to keep this simple.

In [ ]:
demo_hk_system.get_rule("PM").commutative = True

In [ ]:
fond = lang.Item("fond", "L x_e : L y_e : Fond(y)(x)")
ofP = lang.Item("of", "L x_e : x")
sentence3 = julius * (isV * (a * (((gray * cat) * (inP * texas)) * (fond * (ofP * john)))))
sentence3

In [ ]:
sentence3.tree()

The _Composite_ class subclasses _nltk.Tree_, and so supports the things that class does.  E.g. []-based paths:

In [ ]:
parse_tree3 = sentence3.results[0]
parse_tree3[0][1][1].tree()

There is support for traces and indexed pronouns, using the PA rule.  (The implementation may not be what you expect.)

In [ ]:
binder = lang.Binder(23)
binder2 = lang.Binder(5)
t = lang.Trace(23, types.type_e)
t2 = lang.Trace(5)
display(t, t2, binder)

In [ ]:
((t * gray))

In [ ]:
b1 = (binder * (binder2 * (t * (inP * t2))))
b2 = (binder2 * (binder * (t * (inP * t2))))
display(b1, b2)

In [ ]:
b1.trace()

In [ ]:
b1.results[0].tree()

### Composition in tree structures

Some in-progress work: implementing tree-based computation, and top-down/deferred computation

* using nltk Tree objects.
* system for deferred / uncertain types -- basic inference over unknown types
* arbitrary order of composition expansion.  (Of course, some orders will be far less efficient!)

In [ ]:
lang.set_system(lang.hk3_system)

In [ ]:
%%lamb
||gray|| = L x_e : Gray_<e,t>(x)
||cat|| = L x_e : Cat_<e,t>(x)

In [ ]:
t2 = Tree("S", ["NP", "VP"])
t2

In [ ]:
t2 = Tree("S", ["NP", "VP"])
r2 = lang.hk3_system.compose(t2)
r2.tree()
r2.paths()

In [ ]:
Tree = lamb.utils.get_tree_class()
t = Tree("NP", ["gray", Tree("N", ["cat"])])
t

In [ ]:
t2 = lang.CompositionTree.tree_factory(t)
r = lang.hk3_system.compose(t2)
r

In [ ]:
r.tree()

In [ ]:
r = lang.hk3_system.expand_all(t2)
r

In [ ]:
r.tree()

In [ ]:
r.paths()

&nbsp;

## Some future projects, non-exhaustive ##

* complete fragment of Heim and Kratzer
* In general: more fragments!
* extend fragment coverage.  Some interesting targets where interactivity would be useful to understanding:
  * Compositional hamblin semantics (partial)
  * Compositional DRT (partial)
  * QR
* underlying model theory.
* various improvements to the graphics -- trees (d3? graphviz?), interactive widgets, ...
* full latex output (trees in tikz-qtree and so on).

Longer term:

* integration with SymPy (?)
* deeper integration with nltk.
* parsing that makes less use of python `eval`, and is generally less ad-hoc.
  * this is an issue where in principle, a language like Haskell is a better choice than python.  But I think the usability / robustness of python and its libraries has the edge here overall, not to mention ipython notebook...
* toy spatial language system
* side-by-side comparison of e.g. multiple analyses of presupposition projection